In [1]:
import argparse
import os

import torch
import torch.nn as nn


from redunet import *
import evaluate
import load as L
import functional as F
import utils
import plot

In [2]:
parser = argparse.ArgumentParser()
parser.add_argument('--data', type=str, required=True, help='choice of dataset')
parser.add_argument('--arch', type=str, required=True, help='choice of architecture')
parser.add_argument('--samples', type=int, required=True, help="number of samples per update")
parser.add_argument('--tail', type=str, default='', help='extra information to add to folder name')
parser.add_argument('--save_dir', type=str, default='./saved_models/', help='base directory for saving.')
parser.add_argument('--data_dir', type=str, default='./data/', help='base directory for saving.')

_StoreAction(option_strings=['--data_dir'], dest='data_dir', nargs=None, const=None, default='./data/', type=<class 'str'>, choices=None, help='base directory for saving.', metavar=None)

## Arguement Setup

In [16]:
args = parser.parse_args(
['--data', 'mnist2d', '--arch', 'lift2d_channels35_layers20', '--samples', '1000']
)


## CUDA

In [4]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
device

device(type='cuda')

## Model Directory

In [5]:
model_dir = os.path.join(args.save_dir, 
                         'forward',
                         f'{args.data}+{args.arch}',
                         f'samples{args.samples}'
                         f'{args.tail}')
os.makedirs(model_dir, exist_ok=True)
utils.save_params(model_dir, vars(args))
print(model_dir)

./saved_models/forward\mnist2d_2class+lift2d_channels35_layers20\samples1000


## Data

In [11]:
from datasets import mnist

In [18]:
#trainset, testset, num_classes = L.load_dataset(args.data, data_dir=args.data_dir)
trainset, testset, num_classes = mnist.mnist2d_2class(data_dir=args.data_dir)
X_train, y_train = F.get_samples(trainset, args.samples)
X_train, y_train = X_train.to(device), y_train.to(device)
X_train.shape

torch.Size([2000, 1, 28, 28])

In [14]:
print(X_train.shape)
y_train.shape

torch.Size([2000, 1, 28, 28])


torch.Size([2000])

## Architectur

In [17]:
net = L.load_architecture(args.data, args.arch)
net = net.to(device)

In [21]:
with torch.no_grad():
    Z_train = net.init(X_train, y_train)
    losses_train = net.get_loss()
    X_train, Z_train = F.to_cpu(X_train, Z_train)

RuntimeError: CUDA out of memory. Tried to allocate 420.00 MiB (GPU 0; 6.00 GiB total capacity; 3.23 GiB already allocated; 206.88 MiB free; 3.81 GiB reserved in total by PyTorch)


## Saving

In [ ]:
utils.save_loss(model_dir, 'train', losses_train)
utils.save_ckpt(model_dir, 'model', net)

In [ ]:
plot.plot_loss_mcr(model_dir, 'train')